In [1]:
# Run once before notebook 
import os
os.chdir("..")
os.getcwd()

'/Users/kaancaylan/Desktop/HEC/mckinsey_hackathon/methane-hackathon'

In [2]:
from utils.dataloader import LoadData
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers
import datetime
from sklearn.model_selection import GroupShuffleSplit

In [3]:

data = LoadData(metadata_path="data/train_data/metadata.csv",
                image_data_path="data/train_data/")

X_train_aug, X_test, y_train_aug, y_test = data.prep_data(normalize=True, augment=True)

/Users/kaancaylan/.pyenv/versions/3.9.6/envs/myenv/lib/python3.9/site-packages/rasterio/__init__.py:304: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


In [4]:
# Set the random seed for reproducibility
tf.random.set_seed(1234)

# Define the CNN model
model = tf.keras.Sequential([
    # Convolutional layers
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    # Flatten the output for fully connected layers
    layers.Flatten(),
    
    # Fully connected layers
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),  
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')  # Use 'sigmoid' for binary classification
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',  # Use 'binary_crossentropy' for binary classification
              metrics=['accuracy'])

# Print the model summary to view the architecture
model.summary()

# Define early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', 
    patience=5,           
    restore_best_weights=True)

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 31, 31, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 14, 14, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 6, 6, 128)         0

In [5]:
batch_size = 32
epochs = 100
model.fit(X_train_aug, y_train_aug, 
          epochs=epochs, 
          batch_size=batch_size, 
          validation_split=0.2,
          callbacks=[early_stopping, tensorboard_callback])

Epoch 1/100
260/260 [==============================] - 8s 29ms/step - loss: 0.5214 - accuracy: 0.7314 - val_loss: 0.3727 - val_accuracy: 0.8308
Epoch 2/100
260/260 [==============================] - 7s 29ms/step - loss: 0.3289 - accuracy: 0.8659 - val_loss: 0.2224 - val_accuracy: 0.9115
Epoch 3/100
260/260 [==============================] - 8s 29ms/step - loss: 0.2100 - accuracy: 0.9232 - val_loss: 0.1611 - val_accuracy: 0.9298
Epoch 4/100
260/260 [==============================] - 7s 29ms/step - loss: 0.1134 - accuracy: 0.9597 - val_loss: 0.1170 - val_accuracy: 0.9548
Epoch 5/100
260/260 [==============================] - 8s 29ms/step - loss: 0.0771 - accuracy: 0.9714 - val_loss: 0.1206 - val_accuracy: 0.9543
Epoch 6/100
260/260 [==============================] - 8s 30ms/step - loss: 0.0536 - accuracy: 0.9830 - val_loss: 0.0560 - val_accuracy: 0.9793
Epoch 7/100
260/260 [==============================] - 8s 29ms/step - loss: 0.0531 - accuracy: 0.9829 - val_loss: 0.0487 - val_accuracy:

In [10]:
%tensorboard --logdir logs/fit

UsageError: Line magic function `%tensorboard` not found.


# Concatenate External features

In [ ]:



input_1d = tf.keras.layers.Input(shape=(1,))  # Adjust the shape as needed

# Concatenate the flattened output and the 1-dimensional input
concatenated = tf.keras.layers.concatenate([model.output, input_1d])

# Add fully connected layers after concatenation
concatenated = layers.Dense(128, activation='relu')(concatenated)
concatenated = layers.Dropout(0.5)(concatenated)
concatenated = layers.Dense(64, activation='relu')(concatenated)
concatenated = layers.Dropout(0.5)(concatenated)
output_layer = layers.Dense(1, activation='sigmoid')(concatenated)

# Create the model
combined_model = tf.keras.Model(inputs=[model.input, input_1d], outputs=output_layer)
